In [20]:
using StatsBase
using RDatasets
using StatsPlots

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1186


## 1. K-means

In [2]:
function euclidean(sourcePoint, destPoint)
    sum = 0
    for i in 1:length(sourcePoint)
        sum += (destPoint[i] - sourcePoint[i]) ^ 2
    end
    dist = sqrt(sum)
    return dist
end

euclidean (generic function with 1 method)

In [66]:
function kmeans(X, k, max_iter = 100, threshold = 0.001)

    centroids = convert(Array{Float64}, X[sample(axes(X, 1), k; replace = false, ordered = true), :])
    new_centroids = copy(centroids)
    cluster_ids = zeros(size(X,1))
    
    for iter in 1:max_iter
        for i in 1:size(X, 1)
            p = X[i,:]
            distances = [[euclidean(p, centroids[j,:]) j] for j = 1:k]
            sort!(distances, by = x -> x[1])
            cluster_ids[i] = distances[1][2]
        end
        
        
        for cluster_id in 1:size(centroids, 1)
            if size([X[i,:] for i in 1:size(X,1) if cluster_ids[i] == cluster_id])[1] != 0
                new_centroids[cluster_id,:] = mean(X[i,:] for i in 1:size(X,1) if cluster_ids[i] == cluster_id)
            end
        end
                    
        center_change = sum(euclidean(centroids[i,:], new_centroids[i,:]) for i in 1:k)

        centroids = copy(new_centroids)
                    
        if center_change < threshold
            break
        end
    end
    
    return centroids, cluster_ids
end

kmeans (generic function with 3 methods)

## 2. RBF-Network

In [55]:
function RBFNetwork(X_train, Y_train, k, r)
    
    # get centroids with kmeans
    centroids = kmeans(X_train, k)[1]
    
    G = zeros(size(X_train)[1], k+1)
    
    for i in 1:size(X_train)[1]
        G[i,1] = 1
        for j in 1:k
            dist = (sum((centroids[j,k] - X_train[i,k])^2 for k in 1:size(X_train,2)))^0.5
            G[i,j+1] = exp(-((dist/r)^2)/2)
        end
    end
    
    w = inv(transpose(G)*G)*transpose(G)*Y_train
    return centroids, w
end

RBFNetwork (generic function with 1 method)

## 3. Erro

In [85]:
function error(X_test, Y_test, centroids, w, k, r)
    perc = 0
    sum_error = 0
    for i in 1:size(X_test)[1]
        g = ones(k+1)
        for j in 1:k
            dist = (sum((centroids[j,k] - X_test[i,k])^2 for k in 1:size(X_test,2)))^0.5
            g[j+1] = exp(-((dist/r)^2)/2)
        end
        sum_error += abs(Y_test[i]-(transpose(w)*g)[1])
        if abs(Y_test[i]-(transpose(w)*g)[1]) > Y_test[i]*0.1
            perc += 1
        end
    end
    return sum_error, perc
end

error (generic function with 1 method)

## 4. Cross Validation

In [57]:
function shufflerows(a::AbstractMatrix)
    n = size(a, 1)
    ord = sample(1:n, n, replace = false)
    a[ord,:]
end

shufflerows (generic function with 1 method)

In [81]:
function cross_validation(X_train, Y_train, k, q = -1)
    if q == -1
        q = [i for i in 1:10]
    end
    
    #rows = [i for i in 1:size(X_train, 1)]
    #rows = shufflerows(rows)
    
    X_train = shufflerows(X_train)
    
    size_train = size(X_train, 1)
    size_part = convert(Int, floor(size_train/length(q)))
    
    min_error = 100000000000000000000 
    best_r = []
    sum_error = 0
    best_perc = 0
    quant_t = 0
    
    for part in 1:length(q)
        if part != length(q)
            elements_part = [x for x in 1:size_train if x<=(part-1)*size_part || x>part*size_part]
            
            X_test_aux = X_train[(part-1)*size_part+1:part*size_part,:]
            X_train_aux = X_train[elements_part,:]
                        
            Y_test_aux = Y_train[(part-1)*size_part+1:part*size_part,:]
            Y_train_aux = Y_train[elements_part,:]
        else          
            X_test_aux = X_train[(part-1)*size_part+1:end, :]
            X_train_aux = X_train[1:(part-1)*size_part,:]
                                    
            Y_test_aux = Y_train[(part-1)*size_part+1:end, :]
            Y_train_aux = Y_train[1:(part-1)*size_part,:]             
        end
        
        quant_t += length(Y_train_aux)
        centroids, w = RBFNetwork(X_train_aux, Y_train_aux, k, q[part])
                
        e, perc = error(X_test_aux, Y_test_aux, centroids, w, k, q[part])
        best_perc += perc

        if e < min_error
            min_error = e
            best_r = q[part]
        end
                    
        sum_error += e
    end
    return best_r, sum_error/length(q), 1-best_perc/quant_t
end

cross_validation (generic function with 2 methods)

## 5. Teste

In [59]:
Data = RDatasets.datasets("Ecdat");
display(sort!(Data, :Dataset));

,Package,Dataset,Title,Rows
,String,String,String,Int64
1,Ecdat,Accident,Ship Accidents,40
2,Ecdat,Airline,Cost for U.S. Airlines,90
3,Ecdat,Airq,Air Quality for Californian Metropolitan Areas,30
4,Ecdat,Benefits,Unemployement of Blue Collar Workers,4877
5,Ecdat,Bids,Bids Received By U.S. Firms,126
6,Ecdat,BudgetFood,Budget Share of Food for Spanish Households,23972
7,Ecdat,BudgetItaly,Budget Shares for Italian Households,1729
8,Ecdat,BudgetUK,Budget Shares of British Households,1519
9,Ecdat,Bwages,Wages in Belgium,1472


In [60]:
Computers = dataset("Ecdat", "Computers")

,Price,Speed,HD,RAM,Screen,CD,Multi,Premium,Ads
,Int32,Int32,Int32,Int32,Int32,Categorical…,Categorical…,Categorical…,Int32
1,1499,25,80,4,14,no,no,yes,94
2,1795,33,85,2,14,no,no,yes,94
3,1595,25,170,4,15,no,no,yes,94
4,1849,25,170,8,14,no,no,no,94
5,3295,33,340,16,14,no,no,yes,94
6,3695,66,340,16,14,no,no,yes,94
7,1720,25,170,4,14,yes,no,yes,94
8,1995,50,85,2,14,no,no,yes,94
9,2225,50,210,8,14,no,no,yes,94


In [133]:
C = convert(Matrix{Float64}, Computers[completecases(Computers), 1:5]);

In [134]:
size(C)

(6259, 5)

In [135]:
C = shufflerows(C);

In [136]:
X_train = C[1:5259, 2:5]
Y_train = C[1:5259, 1];

In [137]:
X_test = C[5230:6259, 2:5]
Y_test = C[5230:6259, 1];

In [146]:
q = [x for x in 15:100]
k = 25
r, in_sample_error, perc = cross_validation(X_train, Y_train, k, q)

(25, 28670.308139564135, 0.9914767960806684)

In [147]:
centroids, w = RBFNetwork(X_train, Y_train, k, r)

([69.4321 341.461 7.92857 14.9429; 59.3571 723.571 13.7143 15.1619; … ; 39.2128 339.966 6.64531 14.7712; 50.0 424.471 6.57692 14.6538], [2616.99, 172.175, -272.689, -1327.27, 179.607, -5.88367, -328.621, -35.6197, -548.824, 1709.91  …  2322.28, -891.45, 804.753, 114.537, -781.98, -2675.2, 147.02, 1417.6, -642.285, -1340.52])

In [148]:
erro_absoluto, pontos_errados = error(X_test, Y_test, centroids, w, k, r)

(368773.2919243612, 620)

In [149]:
1-pontos_errados/length(Y_test)

0.3980582524271845